## 2. Image Pipeline

The goals / steps of this project are the following:

* Apply a distortion correction to raw images.
* Undistort image
* Unwrap undistorted image
* Grad binary unwraped image
* HLS from grad binary image

---
## Globals and libs

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import pickle
%matplotlib qt

# get images
images = glob.glob('test_images/*.jpg')

# get mtx, dist
dist_pickle = pickle.load( open( "examples/wide_dist_pickle.p", "rb" ) )
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]



FileNotFoundError: [Errno 2] No such file or directory: 'examples/wide_dist_pickle.p'

## Functions

In [ ]:
def cal_undistort(img):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist


def warper(img, src, dst):
    img_size = (img.shape[1], img.shape[0])
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_NEAREST)
    return warped


def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output


def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # Return the binary image
    return binary_output


def abs_sobel_thresh(img, orient='x', thresh_min=0, thresh_max=255):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Return the result
    return binary_output




In [ ]:
## Example of distortion-corrected image

In [ ]:
img = cv2.imread('test_images/straight_lines1.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
undistort = cal_undistort(gray)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(gray)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(undistort)
ax2.set_title('Undistorted Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
plt.savefig('output_images/undistorted_image.jpg')


In [ ]:
## Unwraped image

In [ ]:
h,w = undistort.shape[:2]

src = np.float32([(575,464),
                  (707,464), 
                  (258,682), 
                  (1049,682)])

dst = np.float32([(450,0),
                  (w-450,0),
                  (450,h),
                  (w-450,h)])

unwarped = warper(undistort, src, dst)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(undistort)
ax1.set_title('Undistorted Image', fontsize=50)
ax2.imshow(unwarped)
ax2.set_title('Unwarped Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
plt.savefig('output_images/unwarped_image.jpg')

In [ ]:
## Color Thresholds

In [ ]:
R = unwarped[:,:,0]
G = unwarped[:,:,1]
B = unwarped[:,:,2]
thresh = (200, 255)
binary = np.zeros_like(R)
binary[(R > thresh[0]) & (R <= thresh[1])] = 1

# Save binary
plt.imshow(binary, cmap='gray')
# plt.savefig('output_images/binary.jpg')

# Show pictures
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(unwarped)
ax1.set_title('Unwraped Image', fontsize=50)
ax2.imshow(binary, cmap='gray')
ax2.set_title('Grad Binary', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
# plt.savefig('output_images/grad_binary.jpg')

hls = cv2.cvtColor(unwarped, cv2.COLOR_RGB2HLS)
H = hls[:,:,0]
L = hls[:,:,1]
S = hls[:,:,2]
thresh = (15, 100)
h_binary = np.zeros_like(H)
h_binary[(H > thresh[0]) & (H <= thresh[1])] = 1

l_binary = np.zeros_like(L)
l_binary[(L > thresh[0]) & (L <= thresh[1])] = 1

thresh = (90, 255)
s_binary = np.zeros_like(S)
s_binary[(S > thresh[0]) & (S <= thresh[1])] = 1


f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(24, 9))
f.tight_layout()
ax1.imshow(unwarped)
ax1.set_title('Unwraped Image', fontsize=30)

ax2.imshow(h_binary, cmap='gray')
ax2.set_title('H Binary', fontsize=30)

ax3.imshow(l_binary, cmap='gray')
ax3.set_title('L Binary', fontsize=30)

ax4.imshow(s_binary, cmap='gray')
ax4.set_title('S Binary', fontsize=30)

plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
# plt.savefig('output_images/hls_binary.jpg')

In [ ]:
## Sobel Examples

In [ ]:
sobel_x = abs_sobel_thresh(unwarped, orient='x', thresh_min=20, thresh_max=100)
sobel_y = abs_sobel_thresh(unwarped, orient='y', thresh_min=20, thresh_max=100)
sobel_mag = mag_thresh(unwarped, sobel_kernel=3, mag_thresh=(30, 100))
sobel_dir = dir_threshold(unwarped, sobel_kernel=15, thresh=(0.7, 1.3))

f, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1, 5, figsize=(24, 9))
f.tight_layout()
ax1.imshow(unwarped)
ax1.set_title('Unwraped Image', fontsize=20)

ax2.imshow(sobel_x, cmap='gray')
ax2.set_title('Thresholded Gradient x orient', fontsize=20)

ax3.imshow(sobel_y, cmap='gray')
ax3.set_title('Thresholded Gradient y orient', fontsize=20)

ax4.imshow(sobel_mag, cmap='gray')
ax4.set_title('Thresholded Magnitude xy orient', fontsize=20)

ax5.imshow(sobel_dir, cmap='gray')
ax5.set_title('Thresholded Grad. Dir.', fontsize=20)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
plt.savefig('output_images/sobels.jpg')


In [ ]:
## Finding the lanes